In [33]:
import torch
import torch.nn as nn
import torch.nn.functional as functional
import torchvision 
from torchvision.datasets import ImageFolder
import torch.optim as optim
import os
from xlwt import Workbook

In [34]:
class Network(nn.Module):
    def __init__(self):
        self.output_size = 10
        
        super(Network, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.pool = nn.MaxPool2d(2)
        self.fc = nn.Linear(400, 120)
        self.fc1 = nn.Linear(120, 84)
        self.fc2 = nn.Linear(84, self.output_size)
        
        
    def forward(self, x):
        x = self.pool(functional.relu(self.conv1(x)))
        x = self.pool(functional.relu(self.conv2(x)))
        x = x.view(-1, 400)
        x = functional.relu(self.fc(x))
        x = functional.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [35]:
def getTrainLoader(batchsize = 24):
    custom_transform = torchvision.transforms.Compose([
        # torchvision.transforms.RandomResizedCrop(size=32,scale=(0.5,1.0)), 
        # torchvision.transforms.RandomPerspective(distortion_scale=0.6,p=1.0),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize((0.5 ), (0.5))
        ])

    train = ImageFolder(root='./trainset', transform=custom_transform)
    trainloader = torch.utils.data.DataLoader(
        train,
        batch_size = batchsize,
        shuffle = False
    )

    # print('Number of training images is {}'.format(len(train))) for debug
    return trainloader

In [36]:
wb = Workbook()
sheet1 = wb.add_sheet('Sheet 1')
j = 1
sheet1.write(0, 0, 'epoch')
sheet1.write(0, 1, 'loss')
# sheet1.write(0, 2, 'test_correction')
net = Network()
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr = 1e-2)
running_loss = 0.0
loss1 = 0
trainloader = getTrainLoader(batchsize=24)

for epoch in range(100):
    # correct = calculate_correction()
    # print(correct)
    for i, mydata in enumerate(trainloader, 0):
        inputs, labels = mydata
        optimizer.zero_grad()
        predicted_labels = net(inputs)
        loss = loss_function(predicted_labels, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        loss1 += loss.item()
        if i % 20 == 19:    # print every 20 mini-batches
            print('Epoch: %d, after mini-batch: %5d loss: %.6f' % (epoch + 1, i + 1, running_loss / 20))
            running_loss = 0.0
    sheet1.write(j, 1, loss1)
    sheet1.write(j, 0, j)
    # sheet1.write(j, 2, correct)
    j += 1
    running_loss = 0.0
    loss1 = 0
            
print('Finished training')
torch.save(net.state_dict(), './result.pth')
print('The magic is done')
# dataiter = iter(testloader)
# images, labels = dataiter.next()

# # Predict the output using the trained neural network
# outputs = net(images)

# # Normalize the outputs using the Softmax function so that
# # we can interpret it as a probability distribution.
# sm = nn.Softmax(dim=1)      
# sm_outputs = sm(outputs)

# # For each output the prediction with the highest probability
# # is the predicted label
# probs, index = torch.max(sm_outputs, dim=1)
# for p, i in zip(probs, index):
#     print('True label {0}, Predicted label {0} - {1:.6f}'.format(classes[i], p))
if os.path.isfile('train_result.xls'):
    os.remove('train_result.xls')
wb.save('train_result.xls')

Epoch: 1, after mini-batch:    20 loss: 2.104989
Epoch: 1, after mini-batch:    40 loss: 1.688766
Epoch: 1, after mini-batch:    60 loss: 1.522089
Epoch: 1, after mini-batch:    80 loss: 1.440944
Epoch: 1, after mini-batch:   100 loss: 2.186397
Epoch: 2, after mini-batch:    20 loss: 1.167768
Epoch: 2, after mini-batch:    40 loss: 0.777424
Epoch: 2, after mini-batch:    60 loss: 1.439861
Epoch: 2, after mini-batch:    80 loss: 1.142822
Epoch: 2, after mini-batch:   100 loss: 1.345583
Epoch: 3, after mini-batch:    20 loss: 0.625060
Epoch: 3, after mini-batch:    40 loss: 0.448004
Epoch: 3, after mini-batch:    60 loss: 0.689154
Epoch: 3, after mini-batch:    80 loss: 0.813249
Epoch: 3, after mini-batch:   100 loss: 0.786016
Epoch: 4, after mini-batch:    20 loss: 0.374752
Epoch: 4, after mini-batch:    40 loss: 0.251475
Epoch: 4, after mini-batch:    60 loss: 0.443913
Epoch: 4, after mini-batch:    80 loss: 0.572159
Epoch: 4, after mini-batch:   100 loss: 0.486310
Epoch: 5, after mini